In [1]:
import pandas as pd

path = "/content/2024_fb_ads_president_scored_anon.csv"

df = pd.read_csv(path, on_bad_lines='skip', engine='python')  # skip problematic lines
df.head()

,page_id,ad_id,ad_creation_time,bylines,currency,delivery_by_region,demographic_distribution,estimated_audience_size,estimated_impressions,estimated_spend,...,lgbtq_issues_topic_illuminating,military_topic_illuminating,race_and_ethnicity_topic_illuminating,safety_topic_illuminating,social_and_cultural_topic_illuminating,technology_and_privacy_topic_illuminating,womens_issue_topic_illuminating,incivility_illuminating,freefair_illuminating,fraud_illuminating
0,4ff23a48b53d988df50ddfebb0e442a984ab8f94e874ef...,0ddb025b8544e2d58e6977ad417e742a52522b3e1fc1c9...,2024-10-21,Texas Organizing Project PAC,USD,"{'Texas': {'spend': 249, 'impressions': 47499}}","{'female_18-24': {'spend': 28, 'impressions': ...",30000,47499,249,...,0,0,0,0,0,0,0,0,0,0
1,4ff23a48b53d988df50ddfebb0e442a984ab8f94e874ef...,86229868e6bde3661724fe02da93504bb4fb5da8c2550d...,2024-10-18,Texas Organizing Project PAC,USD,"{'Texas': {'spend': 49, 'impressions': 22499}}","{'female_18-24': {'spend': 8, 'impressions': 3...",75000,22499,49,...,0,0,0,0,0,0,0,0,0,0
2,4ff23a48b53d988df50ddfebb0e442a984ab8f94e874ef...,07b5aefc27e872e971f793e49aac38496fa62e484f3928...,2024-10-13,Texas Organizing Project PAC,USD,"{'Texas': {'spend': 149, 'impressions': 32499}}","{'female_18-24': {'spend': 26, 'impressions': ...",75000,32499,149,...,0,0,0,0,0,0,0,0,0,0
3,b9eb7e353e596d5fc99568d4ef77d4b11ced3243537cbd...,c62978153c04116d88ead49379916855f2cb58bf788631...,2024-11-02,NaN,USD,{},{},1000001,499,49,...,0,0,0,0,0,0,0,0,0,0
4,b9eb7e353e596d5fc99568d4ef77d4b11ced3243537cbd...,785e91ef18a5794565af03a6df4e7077fe1d915bfb3402...,2024-11-02,NaN,USD,{},{},1000001,499,49,...,0,0,0,0,0,0,0,0,0,0


In [20]:
import csv
import statistics
from collections import defaultdict, Counter
# Removed pandas import as we are aiming for pure Python processing

def is_number(val):
    """Checks if a value can be converted to a float."""
    try:
        float(val)
        return True
    except (ValueError, TypeError):
        return False

def summarize_column(col_values):
    """Summarizes a list of values, handling both numeric and categorical data."""
    numeric_values = [float(v) for v in col_values if is_number(v)]

    if numeric_values:
        # Summarize numeric data
        summary = {
            "count": len(numeric_values),
            "mean": round(statistics.mean(numeric_values), 2) if numeric_values else 0,
            "min": min(numeric_values) if numeric_values else "N/A",
            "max": max(numeric_values) if numeric_values else "N/A",
        }
        if len(numeric_values) > 1:
            try:
                summary["std_dev"] = round(statistics.stdev(numeric_values), 2)
            except statistics.StatisticsError:
                summary["std_dev"] = "N/A"
        else:
            summary["std_dev"] = "N/A"
        return summary
    else:
        # Summarize categorical data
        # Ensure all values are strings for consistent counting
        col_values_str = [str(v) if v is not None else 'None' for v in col_values]
        return {
            "unique": len(set(col_values_str)),
            "most_common": Counter(col_values_str).most_common(1)[0] if col_values_str else ("N/A", 0)
        }

def summarize_dataset(data_list):
    """Summarizes data provided as a list of dictionaries (rows)."""
    print("\n--- Overall Summary ---")
    if not data_list:
        print("No data to summarize.")
        return

    # Get column names from the first dictionary (assuming consistent keys)
    columns = data_list[0].keys()

    for col in columns:
        # Extract values for the current column across all rows
        col_values = [row.get(col) for row in data_list]
        print(f"\nColumn: {col}")
        for k, v in summarize_column(col_values).items():
            print(f"  {k}: {v}")

def group_by(data_list, keys):
    """Groups data (list of dictionaries) by specified keys."""
    grouped = defaultdict(list)
    for row in data_list:
        try:
            # Create a tuple key from the values of the specified keys
            key = tuple(row.get(k) for k in keys)
            grouped[key].append(row)
        except KeyError as e:
            print(f"Warning: Missing key '{e}' in a row. Skipping row.")
            continue
    return grouped

# === MAIN ===
# The data was loaded into a Polars DataFrame 'df' in a previous cell.
# Convert the Polars DataFrame to a list of dictionaries.
# Use the 'df' variable that exists from the previous successful execution.
data_list = df.to_dicts()


summarize_dataset(data_list)

print("\n--- Grouped by page_id ---")
group1 = group_by(data_list, ["page_id"])
# Iterate through the first 2 groups and summarize
for i, (key, group_data) in enumerate(list(group1.items())):
    if i >= 2: break
    print(f"\nGroup: {key}")
    summarize_dataset(group_data)

print("\n--- Grouped by (page_id, ad_id) ---")
group2 = group_by(data_list, ["page_id", "ad_id"])
# Iterate through the first 2 groups and summarize
for i, (key, group_data) in enumerate(list(group2.items())):
    if i >= 2: break
    print(f"\nGroup: {key}")
    summarize_dataset(group_data)


--- Overall Summary ---

Column: page_id
  unique: 4475
  most_common: ('4d66f5853f0365dba032a87704a634f023d15babde973bb7a284ed8cd2707b2d', 55503)

Column: ad_id
  unique: 246745
  most_common: ('0ddb025b8544e2d58e6977ad417e742a52522b3e1fc1c9d9b61c57148f8d72fc', 1)

Column: ad_creation_time
  unique: 547
  most_common: ('2024-10-27', 8619)

Column: bylines
  unique: 3791
  most_common: ('HARRIS FOR PRESIDENT', 49788)

Column: currency
  unique: 18
  most_common: ('USD', 246599)

Column: delivery_by_region
  unique: 141122
  most_common: ('{}', 30989)

Column: demographic_distribution
  unique: 215622
  most_common: ('{}', 30989)

Column: estimated_audience_size
  count: 246745
  mean: 556462.86
  min: 0.0
  max: 1000001.0
  std_dev: 409864.76

Column: estimated_impressions
  count: 246745
  mean: 45601.53
  min: 499.0
  max: 1000000.0
  std_dev: 136790.77

Column: estimated_spend
  count: 246745
  mean: 1061.29
  min: 49.0
  max: 474999.0
  std_dev: 4992.56

Column: publisher_platform

In [10]:
import pandas as pd

# Load the dataset
df = pd.read_csv("2024_fb_ads_president_scored_anon.csv")

# ===== Overall Descriptive Statistics =====
print("\n--- Descriptive Statistics ---")
print(df.describe(include='all'))

# ===== Unique Counts and Most Frequent Values =====
print("\n--- Unique Count and Most Frequent Values ---")
for col in df.columns:
    print(f"\nColumn: {col}")
    print(f"  Unique values: {df[col].nunique()}")
    if df[col].dtype == 'object':
        print(f"  Most frequent: {df[col].value_counts().idxmax()} ({df[col].value_counts().max()} times)")

# ===== Grouped by 'page_id' =====
numeric_cols = df.select_dtypes(include='number').columns

print("\n--- Grouped by page_id ---")
grouped_page = df.groupby("page_id")[numeric_cols].agg(['count', 'mean', 'min', 'max', 'std'])
print(grouped_page.head(3))

print("\n--- Grouped by (page_id, ad_id) ---")
grouped_page_ad = df.groupby(["page_id", "ad_id"])[numeric_cols].agg(['count', 'mean', 'min', 'max', 'std'])
print(grouped_page_ad.head(3))



--- Descriptive Statistics ---
                                                  page_id  \
count                                              246745   
unique                                               4475   
top     4d66f5853f0365dba032a87704a634f023d15babde973b...   
freq                                                55503   
mean                                                  NaN   
std                                                   NaN   
min                                                   NaN   
25%                                                   NaN   
50%                                                   NaN   
75%                                                   NaN   
max                                                   NaN   

                                                    ad_id ad_creation_time  \
count                                              246745           246745   
unique                                             246745              547   
t

In [11]:
import polars as pl

# Load dataset using Polars
df = pl.read_csv("2024_fb_ads_president_scored_anon.csv")

# Print general summary
print("\n--- Descriptive Statistics ---")
print(df.describe())

# Unique + Most frequent
print("\n--- Unique Count and Most Frequent Values ---")
for col in df.columns:
    print(f"\nColumn: {col}")
    print(f"  Unique: {df[col].n_unique()}")
    try:
        most_common = df.group_by(col).count().sort("count", descending=True)[0, col]
        print(f"  Most Frequent: {most_common}")
    except:
        print("  Most Frequent: N/A")

# Select only numeric columns
numeric_cols = [col for col, dtype in zip(df.columns, df.dtypes) if dtype in [pl.Float64, pl.Int64]]

# Grouped by page_id
print("\n--- Grouped by page_id ---")
grouped_page = df.group_by("page_id").agg([
    pl.count(),
    *[pl.mean(col).alias(f"{col}_mean") for col in numeric_cols],
    *[pl.min(col).alias(f"{col}_min") for col in numeric_cols],
    *[pl.max(col).alias(f"{col}_max") for col in numeric_cols],
    *[pl.std(col).alias(f"{col}_std") for col in numeric_cols],
])
print(grouped_page.head(3))

# Grouped by page_id + ad_id
print("\n--- Grouped by (page_id, ad_id) ---")
grouped_page_ad = df.group_by(["page_id", "ad_id"]).agg([
    pl.count(),
    *[pl.mean(col).alias(f"{col}_mean") for col in numeric_cols],
    *[pl.min(col).alias(f"{col}_min") for col in numeric_cols],
    *[pl.max(col).alias(f"{col}_max") for col in numeric_cols],
    *[pl.std(col).alias(f"{col}_std") for col in numeric_cols],
])
print(grouped_page_ad.head(3))



--- Descriptive Statistics ---
shape: (9, 42)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ statistic ┆ page_id   ┆ ad_id     ┆ ad_creati ┆ … ┆ womens_is ┆ incivilit ┆ freefair_ ┆ fraud_il │
│ ---       ┆ ---       ┆ ---       ┆ on_time   ┆   ┆ sue_topic ┆ y_illumin ┆ illuminat ┆ luminati │
│ str       ┆ str       ┆ str       ┆ ---       ┆   ┆ _illumina ┆ ating     ┆ ing       ┆ ng       │
│           ┆           ┆           ┆ str       ┆   ┆ tin…      ┆ ---       ┆ ---       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆ ---       ┆ f64       ┆ f64       ┆ f64      │
│           ┆           ┆           ┆           ┆   ┆ f64       ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ count     ┆ 246745    ┆ 246745    ┆ 246745    ┆ … ┆ 246745.0  ┆ 246745.0  ┆ 246745.0  ┆ 246745.0 │
│ null_coun ┆ 0         ┆ 0         ┆ 0     

/tmp/ipython-input-11-2947145177.py:16: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  most_common = df.group_by(col).count().sort("count", descending=True)[0, col]


  Most Frequent: 4d66f5853f0365dba032a87704a634f023d15babde973bb7a284ed8cd2707b2d

Column: ad_id
  Unique: 246745
  Most Frequent: c43912b26d065f485324975951e64404d8fa3102e7f5dccbcbed7507d4874845

Column: ad_creation_time
  Unique: 547
  Most Frequent: 2024-10-27

Column: bylines
  Unique: 3791
  Most Frequent: HARRIS FOR PRESIDENT

Column: currency
  Unique: 18
  Most Frequent: USD

Column: delivery_by_region
  Unique: 141122
  Most Frequent: {}

Column: demographic_distribution
  Unique: 215622
  Most Frequent: {}

Column: estimated_audience_size
  Unique: 9
  Most Frequent: 1000001

Column: estimated_impressions
  Unique: 39
  Most Frequent: 499

Column: estimated_spend
  Unique: 44
  Most Frequent: 49

Column: publisher_platforms
  Unique: 9
  Most Frequent: ['facebook', 'instagram']

Column: illuminating_scored_message
  Unique: 26338
  Most Frequent: e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855

Column: illuminating_mentions
  Unique: 278
  Most Frequent: []



/tmp/ipython-input-11-2947145177.py:27: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count(),


shape: (3, 126)
┌────────────┬───────┬────────────┬────────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ page_id    ┆ count ┆ estimated_ ┆ estimated_ ┆ … ┆ womens_is ┆ incivilit ┆ freefair_ ┆ fraud_ill │
│ ---        ┆ ---   ┆ audience_s ┆ impression ┆   ┆ sue_topic ┆ y_illumin ┆ illuminat ┆ uminating │
│ str        ┆ u32   ┆ ize_mean   ┆ s_mean     ┆   ┆ _illumina ┆ ating_std ┆ ing_std   ┆ _std      │
│            ┆       ┆ ---        ┆ ---        ┆   ┆ tin…      ┆ ---       ┆ ---       ┆ ---       │
│            ┆       ┆ f64        ┆ f64        ┆   ┆ ---       ┆ f64       ┆ f64       ┆ f64       │
│            ┆       ┆            ┆            ┆   ┆ f64       ┆           ┆           ┆           │
╞════════════╪═══════╪════════════╪════════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ dc14194647 ┆ 965   ┆ 278145.753 ┆ 34334.7512 ┆ … ┆ 0.0       ┆ 0.49426   ┆ 0.0       ┆ 0.0       │
│ 0ce301b7af ┆       ┆ 368        ┆ 95         ┆   ┆           ┆           

/tmp/ipython-input-11-2947145177.py:38: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count(),


shape: (3, 127)
┌────────────┬────────────┬───────┬────────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ page_id    ┆ ad_id      ┆ count ┆ estimated_ ┆ … ┆ womens_is ┆ incivilit ┆ freefair_ ┆ fraud_ill │
│ ---        ┆ ---        ┆ ---   ┆ audience_s ┆   ┆ sue_topic ┆ y_illumin ┆ illuminat ┆ uminating │
│ str        ┆ str        ┆ u32   ┆ ize_mean   ┆   ┆ _illumina ┆ ating_std ┆ ing_std   ┆ _std      │
│            ┆            ┆       ┆ ---        ┆   ┆ tin…      ┆ ---       ┆ ---       ┆ ---       │
│            ┆            ┆       ┆ f64        ┆   ┆ ---       ┆ f64       ┆ f64       ┆ f64       │
│            ┆            ┆       ┆            ┆   ┆ f64       ┆           ┆           ┆           │
╞════════════╪════════════╪═══════╪════════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 4d66f5853f ┆ 0008eb52fb ┆ 1     ┆ 750000.0   ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│ 0365dba032 ┆ cd9259cd8c ┆       ┆            ┆   ┆           ┆           

FB Posts
